In [1]:
%run ./common.ipynb
run ='ALL'
if run == 'SOLE':
    partitionCnt=1
    filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/SOLE/'
else:
    partitionCnt=30
    filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/AllProjects/'

# filepath='./output/'


datestr = ""

g_Df = pd.DataFrame (columns=['key','FixVersion', 
 'Priority',
 'Acceptance Criteria',
 'Severity Level',
 'Labels',
 'EpicLink',
 'Issuelinks',
 'Status',
 'Components',
 'IssueCreator',
 'creator.emailAddress',
 'creator.displayName',
#  'SubTasks',
 'Reporter.name',
 'reporter.emailAddress',
 'reporter.displayName',
 'issueType',
 'projectKey',
 'projectName',
 'created_ts',
 'updated_ts',
 'ParentLink',
 'summary',
 'StoryPoint',
 'Sprint', 
 'AssigneeName',
 'AssigneeEmailAddress',
 'AssigneeDisplayName',
 'ResolutionDescription',
 'ResolutionName',
 'parent.Summary',
 'parent.key',
 'parent.status',
 'parent.priority',
 'Defect Type',
 'Test Cycle',
 'Bug Severity',
 'Sub-track',
 'RICE Object Type',
 'Complexity',
 'Scope',
 'Baseline Scope',
 'Sprint_state',
 'Sprint_name',
 'Sprint_startDate',
 'Sprint_endDate',
 'Sprint_completeDate',        
 'Sprint_activatedDate',
 'Sprint_sequence',
 'Sprint_goal',
 'Intial SOW',
 'After Global Design',
 'The Rudy Special',
 'Changelog',
 'CT_leadTime',
 'CT_development',
 'CT_discovery',
 'CT_deployment',
 'CT_cancelled',
 'Changelog_Rec'
 ])

discovery_state = ["To Do",
            "Backlog",
            "Not Started",
            "Ready",
            "Refinement",
            "Design",
            "New",
            "Drafting",
            "In Review",
            "Pending PO Review",
            "Open",
            "Pending"
            ]
development_state = ['In Progress',
                'On Hold',
                'Rework',
                'Fail',
                'Build',
                'Testing',
                'Reopen',
                'Work In Progress']
deployment_state = ['Ready For Approval',
                'Approved',
                'Done',
                'Delayed',
                'Need Approval',
                'PO Acceptance',
                'Resolved',
                'Closed Complete']

cancel_state = ['Cancelled',
        'Closed - Not Doing',
        'Closed Skipped',
        'Closed Incomplete']


close_state = ['Cancelled',
        'Closed - Duplicate',
        'Closed - Inactivity',
        'Closed - Not Doing',
        'Closed Skipped',
        'Closed Incomplete','Closed','Done',
        'Resolved',
        'Closed Complete']

pd.set_option('display.max_columns', None)


In [2]:
global logger
logger = setup_logger()
logger.info("=======================================")
logger.info("Script start accumulating data from JIRA")

In [3]:
def data_clean(dataframe):
    if dataframe is not None and not dataframe.empty:
       dataframe= dataframe.filter(['key','fields.fixVersions',
                    'fields.priority.name',
                    'fields.customfield_31601',
                    'fields.customfield_12402.value',
                    'fields.labels',
                    'fields.customfield_10006',
                    # 'fields.issuelinks',
                    'fields.status.name',
                    'fields.components',
                    'fields.creator.name',
                    'fields.creator.emailAddress',
                    'fields.creator.displayName',
                    # 'fields.subtasks',
                    'fields.reporter.name',
                    'fields.reporter.emailAddress',
                    'fields.reporter.displayName',
                    'fields.issuetype.name',
                    'fields.project.key',
                    'fields.project.name',
                    'fields.created',
                    'fields.updated',
                    'fields.customfield_12823',
                    'fields.summary',
                    'fields.customfield_10002',
                    'fields.customfield_10004',
                    'fields.assignee.name',
                    'fields.assignee.emailAddress',
                    'fields.assignee.displayName',
                    'fields.resolution.description',
                    'fields.resolution.name',
                    'fields.parent.fields.summary',
                    'fields.parent.key',
                    'fields.parent.fields.status.name',
                    'fields.parent.fields.priority.name',
                    'fields.customfield_32101.value',
                    'fields.customfield_32302.value',
                    'fields.customfield_11605.value',
                    'fields.customfield_32901',
                    # 'fields.customfield_16902' ,
                    'fields.customfield_32900.value' ,
                    'fields.customfield_27503' ,
                    'fields.customfield_16902.value',
                    'fields.customfield_33000',
                    'changelog.histories']).rename(columns={'fields.fixVersions': 'FixVersion', 
                                  'fields.priority.name':'Priority',
                                  'fields.customfield_31601': 'Acceptance Criteria',
                                  'fields.customfield_12402.value':'Severity Level',
                                  'fields.labels':'Labels',
                                  'fields.customfield_10006':'EpicLink',
                                #   'fields.issuelinks': 'Issuelinks',
                                  'fields.status.name': 'Status',
                                  'fields.components':'Components',
                                  'fields.creator.name':'IssueCreator',
                                  'fields.creator.emailAddress':'creator.emailAddress',
                                  'fields.creator.displayName': 'creator.displayName',
                                  # 'fields.subtasks':'SubTasks',
                                  'fields.reporter.name':'Reporter.name',
                                  'fields.reporter.emailAddress':'reporter.emailAddress',
                                  'fields.reporter.displayName': 'reporter.displayName',
                                  'fields.issuetype.name':'issueType',
                                  'fields.project.key':'projectKey',
                                  'fields.project.name':'projectName',
                                  'fields.created':'created_ts',
                                  'fields.updated':'updated_ts',
                                  'fields.customfield_12823': 'ParentLink',
                                  'fields.summary':'summary',
                                  'fields.customfield_10002':'StoryPoint',
                                  'fields.customfield_10004':'Sprint',                                  
                                  'fields.assignee.name':'AssigneeName',
                                  'fields.assignee.emailAddress':'AssigneeEmailAddress',
                                  'fields.assignee.displayName':'AssigneeDisplayName',
                                  'fields.resolution.description':'ResolutionDescription',
                                  'fields.resolution.name':'ResolutionName',
                                  'fields.parent.fields.summary': 'parent.Summary',
                                  'fields.parent.key': 'parent.key',
                                  'fields.parent.fields.status.name': 'parent.status',
                                  'fields.parent.fields.priority.name':'parent.priority',
                                  'fields.customfield_32101.value':'Defect Type',
                                  'fields.customfield_32302.value':'Test Cycle',
                                  'fields.customfield_11605.value':'Bug Severity',
                                  'fields.customfield_32901':'Sub-track',
                                  'fields.customfield_32900.value' : 'RICE Object Type',
                                  'fields.customfield_27503' : 'Complexity',
                                  'fields.customfield_16902.value':'Scope',
                                  'fields.customfield_33000':'Baseline Scope',
                                  'changelog.histories':'Changelog'
                                  })
    return dataframe


In [4]:
def fetch( project, issueType):
    global fetch_df 
    logger.info(f"Fetch {issueType}...")

    if issueType == 'Story':
        attribute =' &expand=projects.issuetypes.fields,changelog'
    else:
        attribute =' &expand=projects.issuetypes.fields'

    searchQry=f'?jql= project in ({project}) and issueType="{issueType}"'

    # if cond_flg == True: 
    #     searchQry=f"{searchQry} and status not in (Cancelled, Done)  {attribute}"
    # else:
    searchQry=f"{searchQry} {attribute}"

    fetch_df = getDataSet(searchQry)

    return data_clean(fetch_df)    
    

In [5]:
def extractSprintDetails(l_Df):
    l_Df['Sprint'] = l_Df['Sprint'].apply(lambda x: convert_to_key_value(x))

    l_Df['Sprint_state'] = l_Df['Sprint'].apply(lambda x: sprintvalue(x,'state') if x is not None else "NA" )
    l_Df['Sprint_name']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'name') if x is not None else "NA" )
    l_Df['Sprint_startDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'startDate') if x is not None else "NA" )
    l_Df['Sprint_endDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'endDate') if x is not None else "NA" )
    l_Df['Sprint_completeDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'completeDate') if x is not None else "NA" )
    l_Df['Sprint_activatedDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'activatedDate') if x is not None else "NA" )
    # l_Df['Sprint_sequence']= epic_Df['Sprint'].apply(lambda x: sprintvalue(x,'sequence') if x is not None else "NA" )
    # l_Df['Sprint_goal']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'goal') if x is not None else "NA" )
    l_Df= l_Df.drop(columns=['Sprint'])

    return l_Df

In [6]:
def getBaselineLst(element):
   baselinelst =[]
   if element != None:
      if  element != 'NA':
         for idx in element:
            baselinelst.append(idx['value'])
   return baselinelst

In [7]:
def getChangeLog(element):
    changelog_Rec = [] 
    for history in element:
        created_at = history.get('created',None)
        for item in history['items']:
            if item['field'] =='status' :
                changelog_Rec.append({
                    'created_at': created_at,
                    'field': item['field'],
                    'from': item['fromString'],
                    'to': item['toString'],
                })
    return changelog_Rec

In [8]:
def split_list(lst, n):
    """Splits a list into n approximately equal parts."""
    k, m = divmod(len(lst), n)
    
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

In [9]:
def format_timedelta(td):
    """Formats a timedelta object to 'day.hours:min:sec' format."""

    seconds = td.total_seconds()
    days, seconds = divmod(seconds, 86400)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)

    return f"{int(days)}.{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

In [10]:
def convert_to_key_value(string):
    result = {}
   
    if string :
        pairs = string[0].split(',')  # Split string by comma
        # logger.info(pairs)
        idx =1
        for pair in pairs:
            key, value = pair.split('=')  # Split each pair by '='
            result[key.strip()] = value.strip()  # Add to dictionary, stripping whitespace
            idx = idx +1 
            if idx > 9:
                break
    return result

In [11]:
def calculateCycleTime(l_Df):
    cycleTime = {}
    for index, row in l_Df.iterrows():
        # display(row)
        _status_change = pd.json_normalize(row['Changelog_Rec'])
        if len(_status_change) > 0:
            logger.debug(row['Status'])
            if (row['Status'] not in close_state):
                # Get the current time in UTC
                now_utc = datetime.now(pytz.utc)
                # Convert to Eastern Time
                eastern = pytz.timezone('US/Eastern')
                now_est = now_utc.astimezone(eastern)
                new_node = {
                            'created_at': now_est.strftime('%Y-%m-%dT%H:%M:%S.%f%z'),
                             'field': 'status',
                             'from': row['Status'],
                             'to' : 'Active'
                            }
                _status_change.loc[len(_status_change)] = new_node

            newCreate_node = {
                        # 'created_at': row['created_ts'].to_list()[0], 
                            'created_at': row['created_ts'],
                            'field': 'status',
                            'from': 'created',
                            'to' : 'WorkflowState'
                        }
            # _status_change['created_at'] = pd.to_datetime(_status_change['created_at'], format='%Y-%m-%dT%H:%M:%S.%f%z')
           
            # _status_change= _status_change.tolist()
            # _status_change= pd.json_normalize(_status_change[0])
            _status_change.loc[len(_status_change)] = newCreate_node   
            _status_change.sort_values(by=['created_at'],inplace=True)
            _status_change['created_at']= pd.to_datetime(_status_change['created_at'])
            _status_change['time_diff'] = _status_change['created_at'].diff()

            # # _status_change['proj']=proj
            # _status_change = _status_change.reset_index(drop=True)
            # # _status_change.to_csv('./output/transistion_history.csv', mode='a', header=False, index=False)
            # # cycleTime['proj'] = proj
            # # cycleTime['key'] = _status_change['key'][0]
            row['CT_leadTime'] = format_timedelta(_status_change['time_diff'].sum())

            row['CT_discovery'] = format_timedelta(_status_change[_status_change['to'].isin( discovery_state )]['time_diff'].sum())
            
            row['CT_development'] = format_timedelta(_status_change[_status_change['to'].isin( development_state)]['time_diff'].sum())                                        
            row['CT_deployment'] = format_timedelta(_status_change[_status_change['to'].isin(deployment_state)]['time_diff'].sum())
            row['CT_cancelled'] = format_timedelta(_status_change[_status_change['to'].isin(cancel_state )]['time_diff'].sum())   

            # current_status['QA'] = format_timedelta(_status_change[_status_change['from'].isin( ['Testing']) ] [_status_change['to'].isin( ['Deploy', 'Active'])]['time_diff'].sum())                    
            # display(_status_change)
            # display(row[['key','CT_leadTime', 'CT_development','CT_discovery', 'CT_deployment'  ]])
    l_Df= l_Df.drop(columns=['Changelog'])
    return l_Df

In [12]:
# strings = projectList()
# # strings ='"SOLMerch","SOLEFIN"'
# split_string_list = split_list(strings, issueList['partitionCnt'])

In [13]:

issueLists = [
              {'issueType':"Portfolio Initiative",'partitionCnt':partitionCnt},
              {'issueType':"Product Initiative", 'partitionCnt':partitionCnt },
              {'issueType':"Epic",'partitionCnt':partitionCnt},
              {'issueType':"Task",'partitionCnt':partitionCnt},
              {'issueType':"Sub-task",'partitionCnt':partitionCnt},
              {'issueType':"Bug",'partitionCnt':partitionCnt},
              {'issueType':"Incident",'partitionCnt':partitionCnt},
              {'issueType':"Production Defects",'partitionCnt':partitionCnt},
              {'issueType':"Defect",'partitionCnt':partitionCnt},
              {'issueType':"Issue",'partitionCnt':partitionCnt},
              {'issueType':"Test",'partitionCnt':partitionCnt},
              {'issueType':"Story",'partitionCnt':partitionCnt}
             ]

# strings = projectList()
strings =pd.Series(["SOLMerch","SOLEFIN"])

for issueList in issueLists:
    split_string_list = split_list(strings, issueList['partitionCnt'])
    filtered_list = [series for series in split_string_list if not series.empty]
    for partition in filtered_list:
        logger.warning(f"processing %s - %s",partition,issueList['issueType'])
        df=fetch(', '.join(['"{}"'.format(value) for value in partition]),issueList['issueType'])
        if df is not None and not df.empty:
            g_Df = pd.concat([g_Df, df ], ignore_index=True)
        

2024-12-06 23:51:58,081 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Portfolio Initiative
2024-12-06 23:51:58,698 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Product Initiative
2024-12-06 23:51:59,625 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Epic
2024-12-06 23:52:12,177 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Task
2024-12-06 23:52:15,337 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Sub-task
2024-12-06 23:55:05,640 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Bug
2024-12-06 23:55:15,738 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Incident
2024-12-06 23:55:16,020 - root - ERROR - No records found....
2024-12-06 23:55:16,026 - root - WARNING - processing 0    SOLMerch
1     SOLEFIN
dtype: object - Production Defects
2024-12-06 23:55:16,292 - root - ERROR - No records found....
2

In [14]:
###### SOLE PROJECT ###########
# issueLists = [
#               {'issueType':"Portfolio Initiative"},
#               {'issueType':"Product Initiative"},
#               {'issueType':"Epic"},
#               {'issueType':"Story"},
#               {'issueType':"Task"},
#               {'issueType':"Sub-task"},
#               {'issueType':"Bug"},
#               {'issueType':"Incident"},
#               {'issueType':"Production Defects"},
#               {'issueType':"Defect"},
#               {'issueType':"Issue"},
#               {'issueType':"Test"}
#              ]
# projects =["SOLMerch","SOLEFIN"]
# # projects =["SOLMerch"]
# for project in projects:
#     for issueList in issueLists:
#         df=fetch(issueList['issueType'], project)
#         if df is not None and not df.empty:
#             g_Df = pd.concat([g_Df, df ], ignore_index=True)
    

In [15]:
g_Df = extractSprintDetails(g_Df)
g_Df=g_Df.fillna("NA")

g_Df['FixVersion']= g_Df['FixVersion'].apply(lambda x: keyvalue(x,'name') if x is not None else "" )
g_Df['Baseline Scope']=g_Df['Baseline Scope'].apply(lambda x: getBaselineLst(x) )
g_Df['Intial SOW']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 1')] )
g_Df['After Global Design']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 2')] )
g_Df['The Rudy Special']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 3')] )

In [16]:
# g_Df[g_Df['Intial SOW']=='YES'][['Intial SOW','After Global Design','The Rudy Special','Baseline Scope']]

In [17]:
logger.info("Upload file to sharepoint...")

try:
    g_Df[g_Df['issueType'] == 'Portfolio Initiative'].to_csv(os.path.join(filepath, 'Portfolio_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Product Initiative'].to_csv(os.path.join(filepath, 'Product_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Epic'].to_csv(os.path.join(filepath, 'Epic.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Task'].to_csv(os.path.join(filepath, 'Task.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Sub-task'].to_csv(os.path.join(filepath, 'Subtasks.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Test'].to_csv(os.path.join(filepath, 'Test.csv'), index=False)
    g_Df.loc[g_Df['issueType'].isin(['Bug','Epic','Incident','Production Defects','Defect','Issue'])].to_csv(os.path.join(filepath, 'Bug.csv'), index=False)
except Exception as e:  
    logger.error(f"Failed to upload files to {filepath}: {e}")
    pass


logger.info("Files uploaded")


In [18]:
logger.info("Working on Stories...")
g_Df_Story = g_Df[g_Df['issueType'] == 'Story']
g_Df_Story['Changelog_Rec'] = g_Df_Story['Changelog'].apply(lambda x: getChangeLog(x) )

In [19]:
g_Df_Story = calculateCycleTime(g_Df_Story)

In [20]:
g_Df_Story

,key,FixVersion,Priority,Acceptance Criteria,Severity Level,Labels,EpicLink,Issuelinks,Status,Components,IssueCreator,creator.emailAddress,creator.displayName,Reporter.name,reporter.emailAddress,reporter.displayName,issueType,projectKey,projectName,created_ts,updated_ts,ParentLink,summary,StoryPoint,AssigneeName,AssigneeEmailAddress,AssigneeDisplayName,ResolutionDescription,ResolutionName,parent.Summary,parent.key,parent.status,parent.priority,Defect Type,Test Cycle,Bug Severity,Sub-track,RICE Object Type,Complexity,Scope,Baseline Scope,Sprint_state,Sprint_name,Sprint_startDate,Sprint_endDate,Sprint_completeDate,Sprint_activatedDate,Sprint_sequence,Sprint_goal,Intial SOW,After Global Design,The Rudy Special,CT_leadTime,CT_development,CT_discovery,CT_deployment,CT_cancelled,Changelog_Rec
8826,SOLMERCH-4821,NA,Medium,NA,Unassigned,"[MOCK_2, Org_Hierarchy, R1]",NA,NA,To Do,[],v008661,v008661@footlocker.com,Abhinav Sharma - v008661,v008661,v008661@footlocker.com,Abhinav Sharma - v008661,Story,SOLMERCH,SOLE Merch,2024-12-06T09:50:02.000-0600,2024-12-06T09:50:02.000-0600,NA,Mock 2 for MRC-EXT-410 Org Hierarchy Upload,5.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,[],NA,NA,NA,NA,NA,NA,NA,NA,NO,NO,NO,NA,NA,NA,NA,NA,[]
8827,SOLMERCH-4820,NA,Medium,NA,Unassigned,"[MOCK_2, Org_Hierarchy, R1]",NA,NA,To Do,[],v008661,v008661@footlocker.com,Abhinav Sharma - v008661,v008661,v008661@footlocker.com,Abhinav Sharma - v008661,Story,SOLMERCH,SOLE Merch,2024-12-06T09:50:01.000-0600,2024-12-06T09:50:01.000-0600,NA,Mock 2 for MRC-EXT-409 Location List Upload,5.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,[],NA,NA,NA,NA,NA,NA,NA,NA,NO,NO,NO,NA,NA,NA,NA,NA,[]
8828,SOLMERCH-4819,NA,Medium,NA,Unassigned,"[Cost, MOCK_2, R1]",NA,NA,To Do,[],v008661,v008661@footlocker.com,Abhinav Sharma - v008661,v008661,v008661@footlocker.com,Abhinav Sharma - v008661,Story,SOLMERCH,SOLE Merch,2024-12-06T09:50:00.000-0600,2024-12-06T09:50:00.000-0600,NA,Mock 2 for MRC-CNV-244 HTS and Duty for Imports,2.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,[],NA,NA,NA,NA,NA,NA,NA,NA,NO,NO,NO,NA,NA,NA,NA,NA,[]
8829,SOLMERCH-4818,NA,Medium,NA,Unassigned,"[MOCK_2, Pricing, R1]",NA,NA,To Do,[],v008661,v008661@footlocker.com,Abhinav Sharma - v008661,v008661,v008661@footlocker.com,Abhinav Sharma - v008661,Story,SOLMERCH,SOLE Merch,2024-12-06T09:49:59.000-0600,2024-12-06T09:49:59.000-0600,NA,Mock 2 for MRC-CNV-242 Active and Approved Cle...,3.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,[],NA,NA,NA,NA,NA,NA,NA,NA,NO,NO,NO,NA,NA,NA,NA,NA,[]
8830,SOLMERCH-4817,NA,Medium,NA,Unassigned,"[Foundation_Data, MOCK_2, R1]",NA,NA,To Do,[],v008661,v008661@footlocker.com,Abhinav Sharma - v008661,v008661,v008661@footlocker.com,Abhinav Sharma - v008661,Story,SOLMERCH,SOLE Merch,2024-12-06T09:49:58.000-0600,2024-12-06T09:49:58.000-0600,NA,Mock 2 for MRC-CNV-239 Global Tax Setup,2.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,[],NA,NA,NA,NA,NA,NA,NA,NA,NO,NO,NO,NA,NA,NA,NA,NA,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12642,SOLEFIN-178,Release 2,Medium,NA,Unassigned,[],SOLEFIN-60,NA,Done,[],v008698,v008698@footlocker.com,Saloni Suvadarsini - v008698 [X],v008698,v008698@footlocker.com,Saloni Suvadarsini - v008698 [X],Story,SOLEFIN,SOLE Finance,2024-02-28T06:02:02.000-0600,2024-07-03T00:33:43.000-0500,NA,Convert Invoice transaction,NA,u741536,Kevin.Reeder@footlocker.com,Kevin Reeder,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,[],CLOSED,Sprint 3,2024-04-08T15:28:00.000-05:00,2024-05-03T15:28:00.000-05:00,2024-07-26T01:49:32.330-05:00,2024-04-08T05:28:15.302-05:00,NA,NA,NO,NO,NO,64.08:35:30,33.05:05:07,8.23:04:38,22.04:25:43,0.00:00:00,[{'created_at': '2024-03-08T05:06:03.210-0600'...
12643,SOLEFIN-177,Release 2,Medium,NA,Unassigned,[],SOLEFIN-60,NA,Done,[],v008698,v008698@footlocker.com,Saloni Suvad

In [21]:
g_Df_Story[g_Df_Story['issueType'] == 'Story'].to_csv(os.path.join(filepath, 'Story.csv'), index=False)

In [22]:
logger.info("Story Files uploaded")

logger.info("Script ended accumulating data from JIRA")
logger.info("=======================================")

In [23]:

# 15/* * * * *  /usr/bin/python3 /Users/u1002018/src/PAE/JIRA/py/sole_proj.py

In [24]:

# def getLinkedIssues(key):
#     check= epic_Df[epic_Df['key'] == key].Issuelinks.astype(bool)
#     if check.to_list()[0] == True:
#         issueLink_str=epic_Df[epic_Df['key'] == key]['Issuelinks']
#         issueLink_str=issueLink_str.to_list()[0][0]

#         searchQry=f'?jql= issue in linkedIssues("{key}")'
#         searchQry_DS = getDataSet(searchQry)
#         linkedIssue=pd.json_normalize(searchQry_DS)

#         specficfields = linkedIssue[['key','fields.status.name','fields.priority.name','fields.summary']]
#         specficfields=specficfields.rename(columns={'fields.status.name': 'status','fields.priority.name':'priority', 'fields.summary':'summary'})
#         json_specficfields =specficfields.to_json(orient='records', lines=True)
#         # print(json_specficfields)
#         return json_specficfields
#     else:
#         return ""
